In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!unzip -q drive/'My Drive'/dataset-original.zip

In [9]:
!mkdir dataset-validation
!mkdir dataset-validation/cardboard
!mkdir dataset-validation/glass
!mkdir dataset-validation/paper
!mkdir dataset-validation/plastic
!mkdir dataset-validation/metal
!mkdir dataset-validation/trash
!ls
!ls dataset-original/trash

dataset-original  dataset-validation  drive  __MACOSX  sample_data
trash100.jpg  trash121.jpg  trash18.jpg  trash39.jpg  trash5.jpg   trash80.jpg
trash101.jpg  trash122.jpg  trash19.jpg  trash3.jpg   trash60.jpg  trash81.jpg
trash102.jpg  trash123.jpg  trash1.jpg	 trash40.jpg  trash61.jpg  trash82.jpg
trash103.jpg  trash124.jpg  trash20.jpg  trash41.jpg  trash62.jpg  trash83.jpg
trash104.jpg  trash125.jpg  trash21.jpg  trash42.jpg  trash63.jpg  trash84.jpg
trash105.jpg  trash126.jpg  trash22.jpg  trash43.jpg  trash64.jpg  trash85.jpg
trash106.jpg  trash127.jpg  trash23.jpg  trash44.jpg  trash65.jpg  trash86.jpg
trash107.jpg  trash128.jpg  trash24.jpg  trash45.jpg  trash66.jpg  trash87.jpg
trash108.jpg  trash129.jpg  trash25.jpg  trash46.jpg  trash67.jpg  trash88.jpg
trash109.jpg  trash12.jpg   trash26.jpg  trash47.jpg  trash68.jpg  trash89.jpg
trash10.jpg   trash130.jpg  trash27.jpg  trash48.jpg  trash69.jpg  trash8.jpg
trash110.jpg  trash131.jpg  trash28.jpg  trash49.jpg  trash6.jpg  

In [0]:
!shuf -n 101 -e dataset-original/glass/* | xargs -i mv {} dataset-validation/glass/
!shuf -n 144 -e dataset-original/paper/* | xargs -i mv {} dataset-validation/paper/
!shuf -n 53 -e dataset-original/cardboard/* | xargs -i mv {} dataset-validation/cardboard/
!shuf -n 82 -e dataset-original/plastic/* | xargs -i mv {} dataset-validation/plastic/
!shuf -n 60 -e dataset-original/metal/* | xargs -i mv {} dataset-validation/metal/
!shuf -n 37 -e dataset-original/trash/* | xargs -i mv {} dataset-validation/trash/

In [11]:
!ls dataset-original/glass | wc -l
!ls dataset-validation/glass | wc -l

400
101


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np

In [0]:
transformationsTrain = transforms.Compose([transforms.Resize(299), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                           transforms.RandomRotation(degrees = (90, -90)), transforms.ToTensor(), 
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformationsValidation = transforms.Compose([transforms.Resize(299), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
train_set = datasets.ImageFolder("dataset-original", transform = transformationsTrain)
val_set = datasets.ImageFolder("dataset-validation", transform = transformationsValidation)

In [0]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=25, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =25, shuffle=True)

In [16]:
!/opt/bin/nvidia-smi
!ps -aux|grep python

Fri Nov 29 02:40:10 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# !kill -9 127

In [10]:
model = torchvision.models.resnet152(pretrained=True)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 6.600 accuracy: 0.040 itemLoss: 4.548
[1,    10] loss: 2.416 accuracy: 0.760 itemLoss: 1.196
[1,    15] loss: 1.342 accuracy: 0.840 itemLoss: 0.770
[1,    20] loss: 1.405 accuracy: 0.600 itemLoss: 1.197
[1,    25] loss: 1.099 accuracy: 0.760 itemLoss: 0.887
[1,    30] loss: 0.922 accuracy: 0.640 itemLoss: 0.894
[1,    35] loss: 0.762 accuracy: 0.760 itemLoss: 0.710
[1,    40] loss: 0.888 accuracy: 0.640 itemLoss: 1.420
[1,    45] loss: 0.778 accuracy: 0.800 itemLoss: 0.806
[1,    50] loss: 0.659 accuracy: 0.720 itemLoss: 0.638
[1,    55] loss: 0.836 accuracy: 0.640 itemLoss: 1.042
[1,    60] loss: 0.681 accuracy: 0.720 itemLoss: 0.654
[1,    65] loss: 0.599 accuracy: 0.800 itemLoss: 0.650
[1,    70] loss: 0.532 accuracy: 0.760 itemLoss: 0.652
[1,    75] loss: 0.622 accuracy: 0.720 itemLoss: 0.959
[1,    80] loss: 0.668 accuracy: 0.920 itemLoss: 0.322
Accuracy of the network on the validation images: 84.067086 %
[2,     5] loss: 0.369 accuracy: 0.920 itemLoss: 0.244
[2,

In [11]:
model = torchvision.models.densenet161(pretrained=True)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:05<00:00, 21.3MB/s]


[1,     5] loss: 6.294 accuracy: 0.200 itemLoss: 4.541
[1,    10] loss: 2.397 accuracy: 0.680 itemLoss: 1.147
[1,    15] loss: 1.516 accuracy: 0.560 itemLoss: 1.765
[1,    20] loss: 1.264 accuracy: 0.640 itemLoss: 1.122
[1,    25] loss: 0.846 accuracy: 0.800 itemLoss: 0.781
[1,    30] loss: 0.779 accuracy: 0.800 itemLoss: 0.657
[1,    35] loss: 0.915 accuracy: 0.720 itemLoss: 0.882
[1,    40] loss: 0.548 accuracy: 0.760 itemLoss: 0.619
[1,    45] loss: 0.578 accuracy: 0.880 itemLoss: 0.369
[1,    50] loss: 0.638 accuracy: 0.800 itemLoss: 0.740
[1,    55] loss: 0.500 accuracy: 0.800 itemLoss: 0.521
[1,    60] loss: 0.357 accuracy: 0.920 itemLoss: 0.269
[1,    65] loss: 0.552 accuracy: 0.880 itemLoss: 0.314
[1,    70] loss: 0.425 accuracy: 0.880 itemLoss: 0.388
[1,    75] loss: 0.439 accuracy: 0.800 itemLoss: 0.431
[1,    80] loss: 0.331 accuracy: 0.800 itemLoss: 0.416
Accuracy of the network on the validation images: 87.840671 %
[2,     5] loss: 0.378 accuracy: 0.880 itemLoss: 0.410
[2,

In [18]:
model = torchvision.models.inception_v3(pretrained=True, aux_logits=False)
model.cuda()

minibatches = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %f %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')

[1,     5] loss: 6.488 accuracy: 0.040 itemLoss: 5.461
[1,    10] loss: 4.495 accuracy: 0.400 itemLoss: 3.707
[1,    15] loss: 2.944 accuracy: 0.520 itemLoss: 2.339
[1,    20] loss: 2.164 accuracy: 0.400 itemLoss: 2.589
[1,    25] loss: 1.609 accuracy: 0.560 itemLoss: 1.287
[1,    30] loss: 1.475 accuracy: 0.520 itemLoss: 1.576
[1,    35] loss: 1.271 accuracy: 0.640 itemLoss: 1.075
[1,    40] loss: 1.079 accuracy: 0.760 itemLoss: 0.963
[1,    45] loss: 1.322 accuracy: 0.600 itemLoss: 1.548
[1,    50] loss: 0.930 accuracy: 0.560 itemLoss: 0.965
[1,    55] loss: 0.936 accuracy: 0.760 itemLoss: 1.021
[1,    60] loss: 0.760 accuracy: 0.640 itemLoss: 0.916
[1,    65] loss: 0.784 accuracy: 0.840 itemLoss: 0.606
[1,    70] loss: 0.629 accuracy: 0.840 itemLoss: 0.644
[1,    75] loss: 0.781 accuracy: 0.720 itemLoss: 0.906
[1,    80] loss: 0.713 accuracy: 0.640 itemLoss: 1.009
Accuracy of the network on the validation images: 79.035639 %
[2,     5] loss: 0.714 accuracy: 0.560 itemLoss: 1.131
[2,